## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

#### Connect to Database

In [8]:
import psycopg2
import pandas as pd
import config as cfg
from scipy import stats

In [3]:
# Create a connection to the midterm database 
con = psycopg2.connect(database=cfg.DATABASE, 
                       user=cfg.USER, 
                       password=cfg.PASSWORD,
                       host=cfg.HOST, 
                       port=cfg.PORT)
cur = con.cursor()

In [7]:
# get flights data by month from 2018,2019 and save to csv

years = ['2018', '2019']
months = [f'{i:02}'for i in range(1,13)]
for year in years:
    for month in months:
        query = f"SELECT * FROM flights WHERE fl_date LIKE '{year}-{month}%'"

        pandas_response = pd.read_sql_query(query, con)

        pandas_response.to_csv(f'data/{month}_{year}_flights.csv')
        print(f'{month}_{year} complete')
        

01_2018 complete
02_2018 complete
03_2018 complete
04_2018 complete
05_2018 complete
06_2018 complete
07_2018 complete
08_2018 complete
09_2018 complete
10_2018 complete
11_2018 complete
12_2018 complete
01_2019 complete
02_2019 complete
03_2019 complete
04_2019 complete
05_2019 complete
06_2019 complete
07_2019 complete
08_2019 complete
09_2019 complete
10_2019 complete
11_2019 complete
12_2019 complete


In [6]:
pandas_response

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,...,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dup,flights
0,2018-01-01,B6,624,N978JB,12892,LAX,"Los Angeles, CA",12478,JFK,"New York, NY",...,323.0,295.0,2475.0,5.0,0.0,9.0,0.0,15.0,N,1.0
1,2018-01-01,B6,508,N353JB,12478,JFK,"New York, NY",14321,PWM,"Portland, ME",...,86.0,51.0,273.0,NaN,NaN,NaN,NaN,NaN,N,1.0
2,2018-01-01,B6,511,N579JB,11697,FLL,"Fort Lauderdale, FL",10423,AUS,"Austin, TX",...,189.0,161.0,1105.0,8.0,0.0,0.0,0.0,237.0,N,1.0
3,2018-01-01,B6,512,N579JB,10423,AUS,"Austin, TX",11697,FLL,"Fort Lauderdale, FL",...,156.0,136.0,1105.0,205.0,0.0,4.0,0.0,39.0,N,1.0
4,2018-01-01,B6,514,N559JB,12954,LGB,"Long Beach, CA",12478,JFK,"New York, NY",...,301.0,279.0,2465.0,NaN,NaN,NaN,NaN,NaN,N,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601214,2018-01-31,AA,2053,N545UW,11057,CLT,"Charlotte, NC",15304,TPA,"Tampa, FL",...,91.0,74.0,507.0,NaN,NaN,NaN,NaN,NaN,N,1.0
601215,2018-01-31,AA,2053,N545UW,15304,TPA,"Tampa, FL",11057,CLT,"Charlotte, NC",...,103.0,81.0,507.0,NaN,NaN,NaN,NaN,NaN,N,1.0
601216,2018-01-31,AA,2054,N722US,11057,CLT,"Charlotte, NC",14100,PHL,"Philadelphia, PA",...,94.0,71.0,449.0,NaN,NaN,NaN,NaN,NaN,N,1.0
601217,2018-01-31,AA,2055,N971UY,11057,CLT,"Charlotte, NC",13930,ORD,"Chicago, IL",...,136.0,92.0,599.0,NaN,NaN,NaN,NaN,NaN,N,1.0


In [15]:
jan = pd.read_csv('data/01_2019_flights.csv', index_col=0)

# Convert flight date to column to datetime
jan['fl_date'] =  pd.to_datetime(jan['fl_date'], format='%Y/%m/%d')

# Create new column for day of the week ie. 0 = monday
jan['day_of_week'] = jan['fl_date'].dt.dayofweek

jan

In [17]:
jan.dtypes

fl_date                datetime64[ns]
mkt_unique_carrier             object
branded_code_share             object
mkt_carrier                    object
mkt_carrier_fl_num              int64
op_unique_carrier              object
tail_num                       object
op_carrier_fl_num               int64
origin_airport_id               int64
origin                         object
origin_city_name               object
dest_airport_id                 int64
dest                           object
dest_city_name                 object
crs_dep_time                    int64
dep_time                      float64
dep_delay                     float64
taxi_out                      float64
wheels_off                    float64
wheels_on                     float64
taxi_in                       float64
crs_arr_time                    int64
arr_time                      float64
arr_delay                     float64
cancelled                     float64
cancellation_code              object
diverted    

#### **Task 1**: 
Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

In [27]:
week1 = jan[(jan['fl_date'] >= '2019-01-01') & (jan['fl_date'] <= '2019-01-07')]

In [36]:
shapiro_test = stats.shapiro(week1['arr_delay'])
shapiro_test

ShapiroResult(statistic=nan, pvalue=1.0)

In [39]:
week1['arr_delay'].mean()

3.2412014918684893

#### **Task 2**: 
Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

#### **Task 3**: 
Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

#### **Task 4**: 
How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**:
What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

#### **Task 6**: 
How many states cover 50% of US air traffic? 

#### **Task 7**: 
Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: 
When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

#### **Task 9**: 
Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: 
Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?